In [ ]:
from pyspark.sql import SparkSession
from Logger import logger
try:
    appName = "Spark hive operations"
    master = "local"
    spark = SparkSession.builder.master(master).appName(appName).enableHiveSupport().getOrCreate()
except Exception as e:
    logger.error(e)

In [ ]:
df = spark.read.csv("hdfs://localhost:9000/CpuLogData/CpuLogData/*.csv",header=True)
df2 = df.select("user_name","DateTime","keyboard","mouse")
df2.createOrReplaceTempView('myview')

In [ ]:
df2.show()

In [ ]:
df2.createOrReplaceTempView("data")

In [ ]:
# df2.write.mode("overwrite").saveAsTable("New.cpu_logs")
# dfs = spark.sql("use New")
# dfs1 = spark.sql("show tables")
# dfs1.show()
# spark.sql("create table namratha.test as select * from myview")
# dfs2 = spark.sql("select *from cpu_logs")
# dfs2.show()

In [ ]:
try:
    spark.sql("create table default.log_data as select * from data")
except Exception as e:
    logger.info(e)

In [ ]:
from pyhive import hive
import pandas as pd

In [ ]:
conn = hive.Connection(host='localhost',port=10000,database='default')
cur = conn.cursor()

In [ ]:
user_count=pd.read_sql("select user_name, count(user_name) as total_count from log_data group by user_name",conn)
print(user_count)

In [ ]:
active_user_count  = pd.read_sql("select user_name ,count(user_name) as total from log_data where keyboard !=0 or mouse!=0 group by user_name",conn)
print(active_user_count)

In [ ]:
highest_work_hrs=pd.read_sql("SELECT user_name,from_unixtime(round((((count('user_name')*5)*60)/6),2),'HH:mm') as work_hours FROM log_data WHERE keyboard !=0 OR mouse !=0 GROUP BY user_name ORDER BY work_hours desc",conn)
print(highest_work_hrs)

In [ ]:
lowest_work_hrs=pd.read_sql("SELECT user_name,from_unixtime(round((((count(user_name)*5)*60)/6),2),'HH:mm') active_hours FROM log_data WHERE keyboard !=0 OR mouse !=0 GROUP BY user_name ORDER BY active_hours asc",conn)
print(lowest_work_hrs)

In [ ]:
idle_count = pd.read_sql("select user_name ,count(user_name) as total from logs where keyboard=0 and mouse=0 group by user_name",conn)
print(idle_count)

In [ ]:
highest_idle_time = pd.read_sql("SELECT user_name,from_unixtime(round((((count(user_name)*5)*60)/6),2),'HH:mm') idle_hours FROM log_data WHERE keyboard==0 AND mouse==0 GROUP BY user_name ORDER BY idle_hours desc",conn)
print(highest_idle_time)